# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import GridSearchCV

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Message_Categories',engine)
X = df.message.values
Y = df.iloc[:,4:].values

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    text = re.sub(r"[^a-z0-9]"," ",text.lower())   
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stopwords.words("english")]
   
    lemmatizer = WordNetLemmatizer()
    
    output_tokens = []
    
    for token in tokens:
        output_token = lemmatizer.lemmatize(token).strip().lower()
        output_tokens.append(output_token)
        
    return output_tokens

In [5]:
X[2]

'Looking for someone but no name'

In [6]:
tokenize(X[2])

['looking', 'someone', 'name']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))  
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [10]:
y_pred = pipeline.predict(X_test)

In [11]:
def print_scores(y_test,y_pred,columns):
    y_pred_df = pd.DataFrame(y_pred)
    y_test_df = pd.DataFrame(y_test)
    
    results = []

    for cat in range(len(y_pred[0])):
        precision,recall,fscore,support = score(y_test_df[cat],y_pred_df[cat],average='weighted') 
        results.append((columns[cat],precision,recall,fscore))
    return pd.DataFrame(results,columns=('Category','Precision','Recall','fscore'))

In [46]:
print_scores(y_test,y_pred,df.columns[4:])

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Category,Precision,Recall,fscore
0,related,0.791366,0.803631,0.793177
1,request,0.872941,0.880226,0.864438
2,offer,0.990259,0.995117,0.992682
3,aid_related,0.750561,0.751755,0.747473
4,medical_help,0.899657,0.922032,0.895149
5,medical_products,0.939156,0.952243,0.934460
6,search_and_rescue,0.963352,0.971773,0.960522
7,security,0.961612,0.980165,0.970800
8,military,0.961485,0.969637,0.957590
9,child_alone,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f921ddc6f28>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [14]:
parameters = {
    'vect__ngram_range' : ((1,1),(1,2)),
    'clf__estimator__n_estimators': (10,25),
    'clf__estimator__min_samples_leaf': (1,2)
}

cv = GridSearchCV(pipeline,param_grid=parameters)

In [ ]:
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_opt = cv.predict(X_test)
print_scores(y_test,y_pred_opt,df.columns[4:])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

### References

* https://stackoverflow.com/questions/48417867/access-to-numbers-in-classification-report-sklearn